# SIFT CNN with Descriptors

In [1]:
import os
import cv2
import requests
import numpy as np
import scipy.io
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the labels file from GitHub
github_url = "https://raw.githubusercontent.com/RyanS974/RyanS974/main/datasets/hep2cell/labels.mat"
labels_path = tf.keras.utils.get_file("labels.mat", github_url)
labels = scipy.io.loadmat(labels_path)["labels"].flatten()[:10000]

print(f"Number of labels: {len(labels)}")

# Set directories
images_dir = "cells"

def preprocess_image(image_path):
    if not os.path.exists(image_path):
        print(f"Image path does not exist: {image_path}")
        return None
    img = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    if img is None:
        print(f"Failed to load image: {image_path}")
        return None
    img_resized = cv2.resize(img, (96, 96))
    img_normalized = img_resized / 255.0
    img_uint8 = (img_normalized * 255).astype(np.uint8)  # Convert back to uint8 for SIFT
    return img_uint8

# SIFT Descriptor extraction
def extract_sift_descriptors(image):
    if image is None or image.dtype != np.uint8:
        print("Invalid image for SIFT descriptor extraction. Ensure the image is non-empty and of type uint8.")
        return [], np.zeros((1, 128))  # Return empty keypoints and dummy descriptors
    sift = cv2.SIFT_create()
    keypoints, descriptors = sift.detectAndCompute(image, None)
    if descriptors is None:
        print("No descriptors found for the image.")
        descriptors = np.zeros((1, 128))  # Default to a single zero descriptor if none found
    return keypoints, descriptors

# Overlay SIFT descriptors for visualization
def visualize_sift(image, keypoints, max_descriptors=5):
    overlay = cv2.drawKeypoints(image, keypoints[:max_descriptors], None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    plt.imshow(overlay, cmap='gray')
    plt.axis('off')
    plt.show()

# Load images and preprocess
def load_images_and_descriptors(images_dir, num_images=10000):
    images = []
    descriptors_list = []
    for i in range(1, num_images + 1):
        image_path = os.path.join(images_dir, f"{i}.png")
        img = preprocess_image(image_path)
        if img is None:
            print(f"Skipping image {image_path} due to preprocessing issues.")
            continue

        keypoints, descriptors = extract_sift_descriptors(img)

        if descriptors is not None:
            descriptors = StandardScaler().fit_transform(descriptors)  # Normalize descriptors

        images.append(img)
        descriptors_list.append(descriptors)

        # Visualize a few samples
        if i <= 5:  # Visualize the first 5 images with descriptors
            visualize_sift(img, keypoints)

        if i % 1000 == 0:
            print(f"Processed {i} images...")

    return np.array(images), descriptors_list

images, descriptors_list = load_images_and_descriptors(images_dir)

# Ensure the number of images matches the number of labels
if len(images) != len(labels):
    raise ValueError(f"Number of images ({len(images)}) does not match number of labels ({len(labels)}).")

# Pad or truncate descriptors for CNN
max_descriptors = max(len(desc) for desc in descriptors_list)
descriptors_padded = [
    np.pad(desc, ((0, max_descriptors - len(desc)), (0, 0)), mode='constant')[:max_descriptors]
    for desc in descriptors_list
]
descriptors_padded = np.array(descriptors_padded)

# Split datasets
X_images = np.array(images)[..., np.newaxis]  # Add channel dimension
Y = labels

X_train_img, X_temp_img, Y_train, Y_temp = train_test_split(X_images, Y, test_size=0.4, random_state=42)
X_val_img, X_test_img, Y_val, Y_test = train_test_split(X_temp_img, Y_temp, test_size=0.5, random_state=42)

X_train_desc, X_temp_desc = train_test_split(descriptors_padded, test_size=0.4, random_state=42)
X_val_desc, X_test_desc = train_test_split(X_temp_desc, test_size=0.5, random_state=42)

# Combine image and descriptor features
def combine_features(X_images, X_descriptors):
    X_descriptors_reshaped = X_descriptors.reshape(len(X_descriptors), 1, 1, -1)
    X_descriptors_tiled = np.tile(X_descriptors_reshaped, (1, 96, 96, 1))
    X_combined = np.concatenate([X_images, X_descriptors_tiled], axis=-1)
    return X_combined

X_train = combine_features(X_train_img, X_train_desc)
X_val = combine_features(X_val_img, X_val_desc)
X_test = combine_features(X_test_img, X_test_desc)

# Simplest CNN Sequential Model
def build_simple_cnn(input_shape):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')  # Assuming binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Debug information
def debug_shapes():
    print(f"Training data shape: {X_train.shape}")
    print(f"Validation data shape: {X_val.shape}")
    print(f"Test data shape: {X_test.shape}")
    print(f"Max descriptors per image: {max_descriptors}")

debug_shapes()

# Build and train CNN model
model = build_simple_cnn(input_shape=X_train.shape[1:])
model.fit(X_train, Y_train, epochs=5, batch_size=32, validation_data=(X_val, Y_val))

KeyboardInterrupt: 

In [1]:
import os
import cv2
import requests
import numpy as np
import scipy.io
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

# Load the labels file from GitHub
github_url = "https://raw.githubusercontent.com/RyanS974/RyanS974/main/datasets/hep2cell/labels.mat"
labels_path = tf.keras.utils.get_file("labels.mat", github_url)
labels = scipy.io.loadmat(labels_path)["labels"].flatten()

# Load file names from cells2.txt
with open('cells2.txt', 'r') as file:
    file_names = file.read().splitlines()

# Filter the file names to ensure they are within the range 1-10,000
filtered_file_names = [file for file in file_names if int(os.path.basename(file).split('.')[0]) <= 10000]

# Select the first 10,000 images from the filtered dataset
selected_file_names = filtered_file_names[:10000]
selected_labels = labels[:10000]

# Ensure that the number of selected file names matches the number of selected labels
assert len(selected_file_names) == len(selected_labels), "Mismatch between number of selected file names and labels"

# Split the selected file names and labels into training, validation, and testing sets
train_files, temp_files, train_labels, temp_labels = train_test_split(
    selected_file_names, selected_labels, test_size=0.6, random_state=42)

val_files, test_files, val_labels, test_labels = train_test_split(
    temp_files, temp_labels, test_size=0.5, random_state=42)

# Define the target image size
target_size = (96, 96)

# Function to preprocess images: resize, convert to grayscale, and normalize
def preprocess_image_for_sift(image_path, target_size):
    # Read the image from the file
    image = cv2.imread(image_path)
    # Check if the image was read correctly
    if image is None:
        print(f"Error reading image: {image_path}")
        return None
    # Resize the image to the target size
    image_resized = cv2.resize(image, target_size)
    # Convert the image to grayscale
    image_gray = cv2.cvtColor(image_resized, cv2.COLOR_BGR2GRAY)
    # Normalize the pixel values to the range [0, 1]
    image_normalized = image_gray / 255.0
    # Convert to uint8
    image_uint8 = (image_normalized * 255).astype(np.uint8)
    return image_uint8

# Preprocess training images
train_images = [preprocess_image_for_sift(file, target_size) for file in train_files if preprocess_image_for_sift(file, target_size) is not None]

# Preprocess validation images
val_images = [preprocess_image_for_sift(file, target_size) for file in val_files if preprocess_image_for_sift(file, target_size) is not None]

# Preprocess testing images
test_images = [preprocess_image_for_sift(file, target_size) for file in test_files if preprocess_image_for_sift(file, target_size) is not None]

# Convert lists to numpy arrays
train_images = np.array(train_images)
val_images = np.array(val_images)
test_images = np.array(test_images)

# Ensure images are in the correct format for SIFT
train_images = (train_images * 255).astype(np.uint8)
val_images = (val_images * 255).astype(np.uint8)
test_images = (test_images * 255).astype(np.uint8)

# Initialize the SIFT detector
sift = cv2.SIFT_create(nfeatures=1000, contrastThreshold=0.01, edgeThreshold=5, sigma=1.2)

# Function to extract SIFT descriptors from an image
def extract_sift_descriptors(image):
    # Check if the image is not empty and has the correct depth
    if image is None or image.dtype != np.uint8:
        print("Invalid image for SIFT descriptor extraction")
        return None
    # Detect SIFT keypoints and compute descriptors
    keypoints, descriptors = sift.detectAndCompute(image, None)
    return descriptors

# Extract SIFT descriptors for training images
train_descriptors = [extract_sift_descriptors(image) for image in train_images]
train_descriptors = [desc for desc in train_descriptors if desc is not None]

# Extract SIFT descriptors for validation images
val_descriptors = [extract_sift_descriptors(image) for image in val_images]
val_descriptors = [desc for desc in val_descriptors if desc is not None]

# Extract SIFT descriptors for testing images
test_descriptors = [extract_sift_descriptors(image) for image in test_images]
test_descriptors = [desc for desc in test_descriptors if desc is not None]

# Pad or truncate descriptors for CNN
max_descriptors = max(len(desc) for desc in train_descriptors + val_descriptors + test_descriptors)
train_descriptors_padded = [
    np.pad(desc, ((0, max_descriptors - len(desc)), (0, 0)), mode='constant')[:max_descriptors]
    for desc in train_descriptors
]
val_descriptors_padded = [
    np.pad(desc, ((0, max_descriptors - len(desc)), (0, 0)), mode='constant')[:max_descriptors]
    for desc in val_descriptors
]
test_descriptors_padded = [
    np.pad(desc, ((0, max_descriptors - len(desc)), (0, 0)), mode='constant')[:max_descriptors]
    for desc in test_descriptors
]

# Convert to numpy arrays
train_descriptors_padded = np.array(train_descriptors_padded)
val_descriptors_padded = np.array(val_descriptors_padded)
test_descriptors_padded = np.array(test_descriptors_padded)

# Combine image and descriptor features
def combine_features(X_images, X_descriptors):
    X_descriptors_reshaped = X_descriptors.reshape(len(X_descriptors), 1, 1, -1)
    X_descriptors_tiled = np.tile(X_descriptors_reshaped, (1, 96, 96, 1))
    X_combined = np.concatenate([X_images, X_descriptors_tiled], axis=-1)
    return X_combined

X_train = combine_features(train_images, train_descriptors_padded)
X_val = combine_features(val_images, val_descriptors_padded)
X_test = combine_features(test_images, test_descriptors_padded)

# Simplest CNN Sequential Model
def build_simple_cnn(input_shape):
    model = Sequential([
        Conv2D(16, (3, 3), activation='relu', input_shape=input_shape),
        MaxPooling2D((2, 2)),
        Flatten(),
        Dense(64, activation='relu'),
        Dense(1, activation='sigmoid')  # Assuming binary classification
    ])
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    return model

# Debug information
def debug_shapes():
    print(f"Training data shape: {X_train.shape}")
    print(f"Validation data shape: {X_val.shape}")
    print(f"Test data shape: {X_test.shape}")
    print(f"Max descriptors per image: {max_descriptors}")

debug_shapes()

# Build and train CNN model
model = build_simple_cnn(input_shape=X_train.shape[1:])
model.fit(X_train, train_labels, epochs=5, batch_size=32, validation_data=(X_val, val_labels))

KeyboardInterrupt: 